# The Four Pillars of Object-Oriented Programming (OOP)

This notebook covers the four fundamental principles of OOP with Python examples, best practices, and interview-focused content.

## Table of Contents
1. [Encapsulation](#encapsulation)
2. [Inheritance](#inheritance)
3. [Polymorphism](#polymorphism)
4. [Abstraction](#abstraction)
5. [Integration Example](#integration)
6. [Interview Questions](#interview)

<a id='encapsulation'></a>
## 1. Encapsulation

**Definition:** Bundling data (attributes) and methods that operate on the data into a single unit (class), while restricting direct access to some components.

**Purpose:**
- Hide internal implementation details
- Protect data integrity
- Provide controlled access through public interfaces
- Reduce coupling between components

**Python Conventions:**
- Public: `attribute` (normal access)
- Protected: `_attribute` (convention, not enforced)
- Private: `__attribute` (name mangling to `_ClassName__attribute`)

### 1.1 Bad Example: No Encapsulation

In [1]:
# ❌ BAD: Direct attribute access without validation
class BankAccountBad:
    def __init__(self, balance):
        self.balance = balance  # Public attribute

# Problems:
account = BankAccountBad(1000)
print(f"Initial balance: ${account.balance}")

# Direct manipulation - no validation!
account.balance = -500  # ❌ Negative balance allowed!
print(f"Invalid balance: ${account.balance}")

# Anyone can set any value
account.balance = "not a number"  # ❌ Type error possible
print(f"Type error: {account.balance}")

Initial balance: $1000
Invalid balance: $-500
Type error: not a number


### 1.2 Good Example: Proper Encapsulation

In [2]:
# ✅ GOOD: Encapsulated with validation
class BankAccountGood:
    def __init__(self, initial_balance):
        self.__balance = 0  # Private attribute
        self.deposit(initial_balance)  # Use method for validation
    
    def get_balance(self):
        """Public method to read balance"""
        return self.__balance
    
    def deposit(self, amount):
        """Controlled way to add money"""
        if not isinstance(amount, (int, float)):
            raise TypeError("Amount must be a number")
        if amount <= 0:
            raise ValueError("Amount must be positive")
        
        self.__balance += amount
        print(f"✓ Deposited ${amount}. New balance: ${self.__balance}")
    
    def withdraw(self, amount):
        """Controlled way to remove money"""
        if amount > self.__balance:
            raise ValueError("Insufficient funds")
        if amount <= 0:
            raise ValueError("Amount must be positive")
        
        self.__balance -= amount
        print(f"✓ Withdrew ${amount}. New balance: ${self.__balance}")

# Test encapsulation
account = BankAccountGood(1000)
print(f"Current balance: ${account.get_balance()}")

account.deposit(500)
account.withdraw(200)

# Try invalid operations
try:
    account.withdraw(5000)  # More than balance
except ValueError as e:
    print(f"❌ Error: {e}")

try:
    account.deposit(-100)  # Negative amount
except ValueError as e:
    print(f"❌ Error: {e}")

# Private attribute is name-mangled
print(f"\nDirect access to __balance: {hasattr(account, '__balance')}")
print(f"Name-mangled access: {account._BankAccountGood__balance}")

✓ Deposited $1000. New balance: $1000
Current balance: $1000
✓ Deposited $500. New balance: $1500
✓ Withdrew $200. New balance: $1300
❌ Error: Insufficient funds
❌ Error: Amount must be positive

Direct access to __balance: False
Name-mangled access: 1300


### 1.3 Encapsulation with @property Decorator

In [ ]:
# ✅ BEST: Using @property for Pythonic encapsulation
class BankAccountProperty:
    def __init__(self, initial_balance):
        self._balance = initial_balance  # Protected attribute
        self._transaction_count = 0
    
    @property
    def balance(self):
        """Getter - read-only access to balance"""
        return self._balance
    
    @property
    def transaction_count(self):
        """Getter - read-only computed property"""
        return self._transaction_count
    
    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self._balance += amount
        self._transaction_count += 1
        return self._balance
    
    def withdraw(self, amount):
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self._balance -= amount
        self._transaction_count += 1
        return self._balance

# Usage - looks like attribute access but uses methods
account = BankAccountProperty(1000)
print(f"Balance: ${account.balance}")  # Uses @property getter
print(f"Transactions: {account.transaction_count}")

account.deposit(500)
print(f"After deposit: ${account.balance}, Transactions: {account.transaction_count}")

# Cannot directly set balance (no setter defined)
try:
    account.balance = 5000  # ❌ AttributeError
except AttributeError as e:
    print(f"\n✓ Cannot set balance directly: {e}")

### 1.4 Encapsulation with Setter and Deleter

In [ ]:
class Temperature:
    """Temperature class with validation"""
    
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        """Get temperature in Celsius"""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        """Set temperature in Celsius with validation"""
        if value < -273.15:  # Absolute zero
            raise ValueError("Temperature below absolute zero is not possible")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        """Get temperature in Fahrenheit (computed)"""
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        """Set temperature using Fahrenheit"""
        celsius = (value - 32) * 5/9
        self.celsius = celsius  # Uses celsius setter for validation
    
    @celsius.deleter
    def celsius(self):
        """Reset to zero when deleted"""
        print("Resetting temperature to 0°C")
        self._celsius = 0

# Test Temperature class
temp = Temperature(25)
print(f"Temperature: {temp.celsius}°C = {temp.fahrenheit}°F")

# Set using Fahrenheit
temp.fahrenheit = 100
print(f"After setting to 100°F: {temp.celsius:.2f}°C = {temp.fahrenheit}°F")

# Validation works
try:
    temp.celsius = -300  # Below absolute zero
except ValueError as e:
    print(f"\n✓ Validation works: {e}")

# Delete resets
del temp.celsius
print(f"After deletion: {temp.celsius}°C")

### Encapsulation Key Takeaways

**Benefits:**
- ✅ Data validation and integrity
- ✅ Hide implementation details
- ✅ Flexibility to change internal representation
- ✅ Computed properties without changing interface

**Python Best Practices:**
- Use `@property` for Pythonic encapsulation
- Don't over-use private attributes (`__`)
- Protected (`_`) often sufficient
- Document public API clearly

**Common Interview Questions:**
- Difference between `_var` and `__var`?
- Why use `@property` instead of getters/setters?
- How does name mangling work?

<a id='inheritance'></a>
## 2. Inheritance

**Definition:** A mechanism where a new class (child/subclass) derives properties and behavior from an existing class (parent/superclass).

**Purpose:**
- Code reuse
- Establish relationships (is-a)
- Polymorphic behavior
- Specialization of general concepts

**Types:**
- Single inheritance: One parent
- Multiple inheritance: Multiple parents
- Multilevel inheritance: Chain of inheritance

### 2.1 Single Inheritance

In [4]:
# Base class
class Animal:
    """Base class for all animals"""
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def speak(self):
        """Generic speak method"""
        return "Some sound"
    
    def info(self):
        """Common information method"""
        return f"{self.name} is {self.age} years old"

# Derived class
class Dog(Animal):
    """Dog inherits from Animal"""
    
    def __init__(self, name, age, breed):
        super().__init__(name, age)  # Call parent constructor
        self.breed = breed  # Dog-specific attribute
    
    def speak(self):  # Override parent method
        return "Woof! Woof!"
    
    def fetch(self):  # Dog-specific method
        return f"{self.name} is fetching the ball!"

class Cat(Animal):
    """Cat inherits from Animal"""
    
    def __init__(self, name, age, indoor=True):
        super().__init__(name, age)
        self.indoor = indoor
    
    def speak(self):  # Override parent method
        return "Meow!"
    
    def climb(self):  # Cat-specific method
        return f"{self.name} is climbing!"

# Test inheritance
dog = Dog("Buddy", 3, "Golden Retriever")
cat = Cat("Whiskers", 2)

print("=== Dog ===")
print(dog.info())  # Inherited from Animal
print(dog.speak())  # Overridden in Dog
print(dog.fetch())  # Dog-specific
print(f"Breed: {dog.breed}")

print("\n=== Cat ===")
print(cat.info())  # Inherited from Animal
print(cat.speak())  # Overridden in Cat
print(cat.climb())  # Cat-specific

# Check inheritance
print(f"\nIs dog an Animal? {isinstance(dog, Animal)}")
print(f"Is dog a Dog? {isinstance(dog, Dog)}")
print(f"Is dog a Cat? {isinstance(dog, Cat)}")

=== Dog ===
Buddy is 3 years old
Woof! Woof!
Buddy is fetching the ball!
Breed: Golden Retriever

=== Cat ===
Whiskers is 2 years old
Meow!
Whiskers is climbing!

Is dog an Animal? True
Is dog a Dog? True
Is dog a Cat? False


### 2.2 Multiple Inheritance

In [5]:
# Multiple base classes
class Flyable:
    """Mixin for flying capability"""
    
    def fly(self):
        return f"{self.name} is flying!"

class Swimmable:
    """Mixin for swimming capability"""
    
    def swim(self):
        return f"{self.name} is swimming!"

# Single inheritance from Animal
class Duck(Animal, Flyable, Swimmable):
    """Duck can fly and swim"""
    
    def speak(self):
        return "Quack!"

class Penguin(Animal, Swimmable):
    """Penguin can swim but not fly"""
    
    def speak(self):
        return "Noot noot!"

# Test multiple inheritance
duck = Duck("Donald", 2)
penguin = Penguin("Pingu", 1)

print("=== Duck ===")
print(duck.info())
print(duck.speak())
print(duck.fly())   # From Flyable
print(duck.swim())  # From Swimmable

print("\n=== Penguin ===")
print(penguin.info())
print(penguin.speak())
print(penguin.swim())  # From Swimmable
print(f"Can penguin fly? {hasattr(penguin, 'fly')}")

# Method Resolution Order (MRO)
print(f"\nDuck MRO: {[cls.__name__ for cls in Duck.__mro__]}")
print(f"Penguin MRO: {[cls.__name__ for cls in Penguin.__mro__]}")

=== Duck ===
Donald is 2 years old
Quack!
Donald is flying!
Donald is swimming!

=== Penguin ===
Pingu is 1 years old
Noot noot!
Pingu is swimming!
Can penguin fly? False

Duck MRO: ['Duck', 'Animal', 'Flyable', 'Swimmable', 'object']
Penguin MRO: ['Penguin', 'Animal', 'Swimmable', 'object']


### 2.3 Multilevel Inheritance

In [ ]:
# Chain of inheritance
class Vehicle:
    """Base vehicle class"""
    
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model
    
    def info(self):
        return f"{self.brand} {self.model}"

class Car(Vehicle):
    """Car extends Vehicle"""
    
    def __init__(self, brand, model, doors):
        super().__init__(brand, model)
        self.doors = doors
    
    def drive(self):
        return f"Driving the {self.info()}"

class ElectricCar(Car):
    """ElectricCar extends Car"""
    
    def __init__(self, brand, model, doors, battery_capacity):
        super().__init__(brand, model, doors)
        self.battery_capacity = battery_capacity
    
    def charge(self):
        return f"Charging {self.battery_capacity}kWh battery"

# Test multilevel inheritance
tesla = ElectricCar("Tesla", "Model 3", 4, 75)

print(tesla.info())      # From Vehicle (grandparent)
print(tesla.drive())     # From Car (parent)
print(tesla.charge())    # From ElectricCar (self)

# Check inheritance chain
print(f"\nIs ElectricCar a Car? {isinstance(tesla, Car)}")
print(f"Is ElectricCar a Vehicle? {isinstance(tesla, Vehicle)}")
print(f"Inheritance chain: {[cls.__name__ for cls in ElectricCar.__mro__]}")

### 2.4 Super() and MRO in Multiple Inheritance

In [ ]:
# Diamond problem example
class A:
    def __init__(self):
        print("A.__init__")
        self.value_a = "from A"

class B(A):
    def __init__(self):
        print("B.__init__")
        super().__init__()
        self.value_b = "from B"

class C(A):
    def __init__(self):
        print("C.__init__")
        super().__init__()
        self.value_c = "from C"

class D(B, C):
    def __init__(self):
        print("D.__init__")
        super().__init__()
        self.value_d = "from D"

# Create instance and observe MRO
print("Creating D instance:")
d = D()

print(f"\nMRO for D: {[cls.__name__ for cls in D.__mro__]}")
print(f"\nValues: A={d.value_a}, B={d.value_b}, C={d.value_c}, D={d.value_d}")

# Explanation: super() follows MRO, not just parent class
# MRO: D -> B -> C -> A -> object
# So A.__init__ is called only once (at the end of the chain)

Creating D instance:
D.__init__

MRO for D: ['D', 'B', 'C', 'A', 'object']


AttributeError: 'D' object has no attribute 'value_a'

### Inheritance Key Takeaways

**Benefits:**
- ✅ Code reuse (DRY principle)
- ✅ Establish hierarchical relationships
- ✅ Enable polymorphism
- ✅ Extend existing classes

**Best Practices:**
- Favor composition over inheritance (when appropriate)
- Use `super()` for parent class calls
- Keep inheritance hierarchies shallow
- Use mixins for multiple inheritance

**Common Pitfalls:**
- Deep inheritance hierarchies
- Diamond problem (solved by MRO in Python)
- Overuse of inheritance (composition often better)

**Interview Questions:**
- What is the diamond problem?
- How does Python's MRO work?
- When to use composition vs inheritance?

<a id='polymorphism'></a>
## 3. Polymorphism

**Definition:** The ability of different objects to respond to the same interface in different ways. "Many forms" - same interface, different implementations.

**Types in Python:**
- **Method Overriding:** Subclass provides different implementation
- **Duck Typing:** "If it walks like a duck and quacks like a duck, it's a duck"
- **Operator Overloading:** Redefine operators for custom classes

**Purpose:**
- Write flexible, reusable code
- Work with objects through common interface
- Extend behavior without modifying existing code

### 3.1 Method Overriding (Runtime Polymorphism)

In [ ]:
# Polymorphism through inheritance
class Shape:
    """Base shape class"""
    
    def area(self):
        raise NotImplementedError("Subclass must implement area()")
    
    def perimeter(self):
        raise NotImplementedError("Subclass must implement perimeter()")

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14159 * self.radius ** 2
    
    def perimeter(self):
        return 2 * 3.14159 * self.radius

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)

class Triangle(Shape):
    def __init__(self, base, height, side1, side2):
        self.base = base
        self.height = height
        self.side1 = side1
        self.side2 = side2
    
    def area(self):
        return 0.5 * self.base * self.height
    
    def perimeter(self):
        return self.base + self.side1 + self.side2

# Polymorphic function - works with any Shape
def print_shape_info(shape: Shape):
    """Same function works with different shape types"""
    print(f"{shape.__class__.__name__}:")
    print(f"  Area: {shape.area():.2f}")
    print(f"  Perimeter: {shape.perimeter():.2f}")

# Create different shapes
shapes = [
    Circle(5),
    Rectangle(4, 6),
    Triangle(3, 4, 5, 5)
]

# Polymorphism in action - same interface, different behavior
for shape in shapes:
    print_shape_info(shape)
    print()

### 3.2 Duck Typing (Python's Favorite)

In [ ]:
# Duck typing - no inheritance needed!
class Duck:
    def speak(self):
        return "Quack!"
    
    def swim(self):
        return "Duck is swimming"

class Person:
    def speak(self):
        return "Hello!"
    
    def swim(self):
        return "Person is swimming"

class Robot:
    def speak(self):
        return "Beep boop!"
    
    def swim(self):
        return "Robot is malfunctioning in water"

# Polymorphic function using duck typing
def make_it_speak_and_swim(entity):
    """Works with ANY object that has speak() and swim() methods"""
    print(entity.speak())
    print(entity.swim())
    print()

# All three work without sharing a common base class!
entities = [Duck(), Person(), Robot()]

for entity in entities:
    print(f"--- {entity.__class__.__name__} ---")
    make_it_speak_and_swim(entity)

# The Pythonic way: "Ask for forgiveness, not permission"
class Car:
    def drive(self):
        return "Car is driving"

def interact_with(thing):
    """Try to use methods without checking type"""
    try:
        print(thing.speak())
    except AttributeError:
        print(f"{thing.__class__.__name__} cannot speak")
    
    try:
        print(thing.swim())
    except AttributeError:
        print(f"{thing.__class__.__name__} cannot swim")

print("--- Duck ---")
interact_with(Duck())
print("\n--- Car ---")
interact_with(Car())

### 3.3 Operator Overloading

In [ ]:
# Custom class with operator overloading
class Vector:
    """2D Vector with operator overloading"""
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        """String representation"""
        return f"Vector({self.x}, {self.y})"
    
    def __add__(self, other):
        """Addition: v1 + v2"""
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        raise TypeError("Can only add Vector to Vector")
    
    def __sub__(self, other):
        """Subtraction: v1 - v2"""
        if isinstance(other, Vector):
            return Vector(self.x - other.x, self.y - other.y)
        raise TypeError("Can only subtract Vector from Vector")
    
    def __mul__(self, scalar):
        """Scalar multiplication: v * scalar"""
        if isinstance(scalar, (int, float)):
            return Vector(self.x * scalar, self.y * scalar)
        raise TypeError("Can only multiply Vector by scalar")
    
    def __eq__(self, other):
        """Equality: v1 == v2"""
        if isinstance(other, Vector):
            return self.x == other.x and self.y == other.y
        return False
    
    def __abs__(self):
        """Magnitude: abs(v)"""
        return (self.x**2 + self.y**2) ** 0.5
    
    def __bool__(self):
        """Truthiness: bool(v)"""
        return self.x != 0 or self.y != 0

# Test operator overloading
v1 = Vector(3, 4)
v2 = Vector(1, 2)
v3 = Vector(0, 0)

print(f"v1 = {v1}")
print(f"v2 = {v2}")
print()

# Arithmetic operators
print(f"v1 + v2 = {v1 + v2}")
print(f"v1 - v2 = {v1 - v2}")
print(f"v1 * 3 = {v1 * 3}")
print()

# Comparison and other operators
print(f"v1 == v2: {v1 == v2}")
print(f"abs(v1) = {abs(v1):.2f}")
print(f"bool(v1) = {bool(v1)}")
print(f"bool(v3) = {bool(v3)}")

### 3.4 Common Magic Methods for Polymorphism

In [ ]:
# Comprehensive example with many magic methods
class Money:
    """Money class demonstrating polymorphism through magic methods"""
    
    def __init__(self, amount, currency="USD"):
        self.amount = amount
        self.currency = currency
    
    def __str__(self):
        """User-friendly string: str(money)"""
        return f"{self.currency} {self.amount:.2f}"
    
    def __repr__(self):
        """Developer-friendly string: repr(money)"""
        return f"Money({self.amount}, '{self.currency}')"
    
    def __add__(self, other):
        """Addition: money1 + money2"""
        if self.currency != other.currency:
            raise ValueError("Cannot add different currencies")
        return Money(self.amount + other.amount, self.currency)
    
    def __lt__(self, other):
        """Less than: money1 < money2"""
        if self.currency != other.currency:
            raise ValueError("Cannot compare different currencies")
        return self.amount < other.amount
    
    def __le__(self, other):
        """Less than or equal"""
        return self < other or self.amount == other.amount
    
    def __hash__(self):
        """Make hashable for use in sets/dicts"""
        return hash((self.amount, self.currency))
    
    def __float__(self):
        """Convert to float: float(money)"""
        return float(self.amount)
    
    def __int__(self):
        """Convert to int: int(money)"""
        return int(self.amount)

# Test Money class
m1 = Money(100.50)
m2 = Money(50.25)

print(f"str(m1): {str(m1)}")
print(f"repr(m1): {repr(m1)}")
print()

print(f"m1 + m2 = {m1 + m2}")
print(f"m1 < m2: {m1 < m2}")
print(f"m1 > m2: {m1 > m2}")
print()

print(f"float(m1) = {float(m1)}")
print(f"int(m1) = {int(m1)}")
print()

# Can use in sets (because __hash__ is defined)
money_set = {m1, m2, Money(100.50)}  # Duplicate will be removed
print(f"Set of money: {money_set}")
print(f"Set length: {len(money_set)} (duplicate removed)")

### Polymorphism Key Takeaways

**Benefits:**
- ✅ Write flexible, reusable code
- ✅ Same interface, different implementations
- ✅ Open/Closed Principle (extend without modifying)
- ✅ Natural syntax through operator overloading

**Python's Approach:**
- Duck typing is preferred over type checking
- "Easier to ask forgiveness than permission" (EAFP)
- Magic methods enable natural syntax

**Common Magic Methods:**
- `__str__`, `__repr__`: String representation
- `__add__`, `__sub__`, `__mul__`: Arithmetic
- `__eq__`, `__lt__`, `__le__`: Comparison
- `__len__`, `__getitem__`: Container protocol
- `__call__`: Make object callable
- `__enter__`, `__exit__`: Context manager

**Interview Questions:**
- What is duck typing?
- Difference between `__str__` and `__repr__`?
- How to make custom class comparable/sortable?

<a id='abstraction'></a>
## 4. Abstraction

**Definition:** Hiding complex implementation details and showing only essential features. Providing a simplified interface to complex systems.

**Purpose:**
- Reduce complexity
- Define contracts (interfaces)
- Force consistent implementation
- Separate "what" from "how"

**Python Implementation:**
- Abstract Base Classes (ABC)
- `@abstractmethod` decorator
- Protocol classes (Python 3.8+)

### 4.1 Abstract Base Classes (ABC)

In [ ]:
from abc import ABC, abstractmethod

# Abstract base class - cannot be instantiated
class PaymentProcessor(ABC):
    """Abstract interface for payment processing"""
    
    @abstractmethod
    def process_payment(self, amount: float) -> bool:
        """Process payment - must be implemented by subclasses"""
        pass
    
    @abstractmethod
    def refund(self, transaction_id: str, amount: float) -> bool:
        """Refund payment - must be implemented"""
        pass
    
    # Concrete method - shared by all subclasses
    def log_transaction(self, transaction_id: str, amount: float):
        """Common logging functionality"""
        print(f"Transaction {transaction_id}: ${amount:.2f}")

# Concrete implementation
class StripePayment(PaymentProcessor):
    """Stripe payment implementation"""
    
    def process_payment(self, amount: float) -> bool:
        print(f"Processing ${amount} via Stripe")
        # Stripe-specific API calls here
        return True
    
    def refund(self, transaction_id: str, amount: float) -> bool:
        print(f"Refunding ${amount} for transaction {transaction_id} via Stripe")
        return True

class PayPalPayment(PaymentProcessor):
    """PayPal payment implementation"""
    
    def process_payment(self, amount: float) -> bool:
        print(f"Processing ${amount} via PayPal")
        # PayPal-specific API calls here
        return True
    
    def refund(self, transaction_id: str, amount: float) -> bool:
        print(f"Refunding ${amount} for transaction {transaction_id} via PayPal")
        return True

# Test abstraction
stripe = StripePayment()
paypal = PayPalPayment()

# Both implement same interface
for processor in [stripe, paypal]:
    print(f"\n--- {processor.__class__.__name__} ---")
    processor.process_payment(100.00)
    processor.refund("TXN123", 50.00)
    processor.log_transaction("TXN123", 50.00)  # Inherited concrete method

# Cannot instantiate abstract class
print("\n--- Testing Abstract Class Instantiation ---")
try:
    processor = PaymentProcessor()  # ❌ TypeError
except TypeError as e:
    print(f"✓ Cannot instantiate ABC: {e}")

### 4.2 Incomplete Implementation Error

In [ ]:
# Incomplete implementation
class IncompletePayment(PaymentProcessor):
    """Only implements one abstract method"""
    
    def process_payment(self, amount: float) -> bool:
        return True
    
    # Missing refund() implementation!

# Try to instantiate incomplete implementation
try:
    incomplete = IncompletePayment()  # ❌ TypeError
except TypeError as e:
    print(f"✓ Incomplete implementation error:\n  {e}")

### 4.3 Abstract Properties

In [ ]:
from abc import ABC, abstractmethod

class DatabaseConnection(ABC):
    """Abstract database connection interface"""
    
    @property
    @abstractmethod
    def connection_string(self) -> str:
        """Abstract property - must be implemented"""
        pass
    
    @abstractmethod
    def connect(self) -> bool:
        """Establish connection"""
        pass
    
    @abstractmethod
    def execute(self, query: str) -> list:
        """Execute query"""
        pass
    
    @abstractmethod
    def close(self) -> None:
        """Close connection"""
        pass

class PostgreSQLConnection(DatabaseConnection):
    """PostgreSQL implementation"""
    
    def __init__(self, host, port, database):
        self.host = host
        self.port = port
        self.database = database
    
    @property
    def connection_string(self) -> str:
        return f"postgresql://{self.host}:{self.port}/{self.database}"
    
    def connect(self) -> bool:
        print(f"Connecting to {self.connection_string}")
        return True
    
    def execute(self, query: str) -> list:
        print(f"Executing PostgreSQL query: {query}")
        return []
    
    def close(self) -> None:
        print("Closing PostgreSQL connection")

class MongoDBConnection(DatabaseConnection):
    """MongoDB implementation"""
    
    def __init__(self, host, port, database):
        self.host = host
        self.port = port
        self.database = database
    
    @property
    def connection_string(self) -> str:
        return f"mongodb://{self.host}:{self.port}/{self.database}"
    
    def connect(self) -> bool:
        print(f"Connecting to {self.connection_string}")
        return True
    
    def execute(self, query: str) -> list:
        print(f"Executing MongoDB query: {query}")
        return []
    
    def close(self) -> None:
        print("Closing MongoDB connection")

# Test database abstraction
def use_database(db: DatabaseConnection):
    """Works with any database implementation"""
    print(f"Connection string: {db.connection_string}")
    db.connect()
    db.execute("SELECT * FROM users")
    db.close()

print("--- PostgreSQL ---")
pg = PostgreSQLConnection("localhost", 5432, "mydb")
use_database(pg)

print("\n--- MongoDB ---")
mongo = MongoDBConnection("localhost", 27017, "mydb")
use_database(mongo)

### 4.4 Real-World Example: Plugin System

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Any

# Abstract plugin interface
class Plugin(ABC):
    """Base class for all plugins"""
    
    @property
    @abstractmethod
    def name(self) -> str:
        """Plugin name"""
        pass
    
    @property
    @abstractmethod
    def version(self) -> str:
        """Plugin version"""
        pass
    
    @abstractmethod
    def initialize(self, config: Dict[str, Any]) -> None:
        """Initialize plugin with configuration"""
        pass
    
    @abstractmethod
    def execute(self, data: Any) -> Any:
        """Execute plugin functionality"""
        pass
    
    @abstractmethod
    def cleanup(self) -> None:
        """Clean up resources"""
        pass

# Concrete plugins
class LoggingPlugin(Plugin):
    """Plugin for logging data"""
    
    @property
    def name(self) -> str:
        return "Logger"
    
    @property
    def version(self) -> str:
        return "1.0.0"
    
    def initialize(self, config: Dict[str, Any]) -> None:
        self.log_level = config.get('level', 'INFO')
        print(f"Initializing {self.name} v{self.version} with level {self.log_level}")
    
    def execute(self, data: Any) -> Any:
        print(f"[{self.log_level}] {data}")
        return data
    
    def cleanup(self) -> None:
        print(f"Cleaning up {self.name}")

class ValidationPlugin(Plugin):
    """Plugin for data validation"""
    
    @property
    def name(self) -> str:
        return "Validator"
    
    @property
    def version(self) -> str:
        return "2.1.0"
    
    def initialize(self, config: Dict[str, Any]) -> None:
        self.rules = config.get('rules', [])
        print(f"Initializing {self.name} v{self.version} with {len(self.rules)} rules")
    
    def execute(self, data: Any) -> Any:
        print(f"Validating data: {data}")
        # Validation logic here
        return data
    
    def cleanup(self) -> None:
        print(f"Cleaning up {self.name}")

# Plugin manager
class PluginManager:
    """Manages plugin lifecycle"""
    
    def __init__(self):
        self.plugins: list[Plugin] = []
    
    def register(self, plugin: Plugin, config: Dict[str, Any]) -> None:
        """Register and initialize a plugin"""
        plugin.initialize(config)
        self.plugins.append(plugin)
    
    def process(self, data: Any) -> Any:
        """Process data through all plugins"""
        result = data
        for plugin in self.plugins:
            result = plugin.execute(result)
        return result
    
    def shutdown(self) -> None:
        """Clean up all plugins"""
        for plugin in self.plugins:
            plugin.cleanup()

# Test plugin system
manager = PluginManager()

# Register plugins
manager.register(LoggingPlugin(), {'level': 'DEBUG'})
manager.register(ValidationPlugin(), {'rules': ['required', 'email']})

print("\n--- Processing Data ---")
manager.process("Important data")

print("\n--- Shutting Down ---")
manager.shutdown()

### Abstraction Key Takeaways

**Benefits:**
- ✅ Define clear contracts/interfaces
- ✅ Force consistent implementation
- ✅ Hide implementation complexity
- ✅ Enable polymorphism
- ✅ Improve testability (mock implementations)

**When to Use ABC:**
- Define interfaces for multiple implementations
- Ensure subclasses implement required methods
- Create plugin/extension systems
- Design frameworks and libraries

**Best Practices:**
- Keep abstract classes focused (SRP)
- Provide concrete helper methods when appropriate
- Use abstract properties for required attributes
- Document expected behavior clearly

**Interview Questions:**
- What is abstraction in OOP?
- How to create abstract classes in Python?
- Difference between abstraction and encapsulation?
- When to use ABC vs Protocol (Python 3.8+)?

<a id='integration'></a>
## 5. Integration Example: All Principles Together

Let's build a complete example that demonstrates all four OOP principles working together.

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
from typing import List

# ====== ABSTRACTION ======
# Define abstract interface for notification systems
class NotificationService(ABC):
    """Abstract notification service interface"""
    
    @abstractmethod
    def send(self, recipient: str, message: str) -> bool:
        """Send notification - must be implemented"""
        pass
    
    @abstractmethod
    def get_delivery_status(self, notification_id: str) -> str:
        """Check delivery status - must be implemented"""
        pass

# ====== ENCAPSULATION ======
# User class with private attributes and controlled access
class User:
    """User with encapsulated data"""
    
    def __init__(self, username: str, email: str):
        self._username = username  # Protected
        self._email = email
        self.__password_hash = None  # Private
        self._notification_preferences = []  # Protected
    
    @property
    def username(self) -> str:
        return self._username
    
    @property
    def email(self) -> str:
        return self._email
    
    def set_password(self, password: str) -> None:
        """Encapsulated password setting with validation"""
        if len(password) < 8:
            raise ValueError("Password must be at least 8 characters")
        # In real code, use proper hashing (bcrypt, etc.)
        self.__password_hash = hash(password)
    
    def verify_password(self, password: str) -> bool:
        """Verify password without exposing hash"""
        return hash(password) == self.__password_hash
    
    def add_notification_preference(self, service: NotificationService) -> None:
        """Add notification preference"""
        self._notification_preferences.append(service)
    
    def get_notification_services(self) -> List[NotificationService]:
        """Get notification services (read-only access)"""
        return self._notification_preferences.copy()

# ====== INHERITANCE & POLYMORPHISM ======
# Different notification implementations inheriting from base
class EmailNotification(NotificationService):
    """Email notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"📧 Sending email to {recipient}: {message}")
        return True
    
    def get_delivery_status(self, notification_id: str) -> str:
        return "delivered"

class SMSNotification(NotificationService):
    """SMS notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"📱 Sending SMS to {recipient}: {message}")
        return True
    
    def get_delivery_status(self, notification_id: str) -> str:
        return "sent"

class PushNotification(NotificationService):
    """Push notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"🔔 Sending push to {recipient}: {message}")
        return True
    
    def get_delivery_status(self, notification_id: str) -> str:
        return "pending"

# Application class using all principles
class NotificationManager:
    """Manages user notifications using all OOP principles"""
    
    def __init__(self):
        self._users: List[User] = []
    
    def register_user(self, user: User) -> None:
        """Register a user"""
        self._users.append(user)
    
    def notify_user(self, username: str, message: str) -> None:
        """Send notification to user via all their preferred services"""
        user = next((u for u in self._users if u.username == username), None)
        if not user:
            print(f"User {username} not found")
            return
        
        # POLYMORPHISM: Same interface, different implementations
        for service in user.get_notification_services():
            service.send(user.email, message)
    
    def broadcast(self, message: str) -> None:
        """Broadcast to all users"""
        for user in self._users:
            self.notify_user(user.username, message)

# ====== DEMONSTRATION ======
print("=" * 60)
print("OOP PRINCIPLES INTEGRATION DEMO")
print("=" * 60)

# Create notification manager
manager = NotificationManager()

# Create users with ENCAPSULATION
user1 = User("alice", "alice@example.com")
user1.set_password("secure_password_123")
user1.add_notification_preference(EmailNotification())
user1.add_notification_preference(SMSNotification())

user2 = User("bob", "bob@example.com")
user2.set_password("another_secure_pass")
user2.add_notification_preference(PushNotification())

# Register users
manager.register_user(user1)
manager.register_user(user2)

# Test ENCAPSULATION
print("\n--- Testing Encapsulation ---")
print(f"User1 username: {user1.username}")  # Property access
print(f"Password verification: {user1.verify_password('secure_password_123')}")
print(f"Wrong password: {user1.verify_password('wrong')}")

# Test POLYMORPHISM
print("\n--- Testing Polymorphism ---")
print("Notifying alice (has Email + SMS):")
manager.notify_user("alice", "Welcome to our platform!")

print("\nNotifying bob (has Push):")
manager.notify_user("bob", "You have a new message!")

# Test ABSTRACTION
print("\n--- Testing Abstraction ---")
print("All services implement the same interface:")
services = [EmailNotification(), SMSNotification(), PushNotification()]
for service in services:
    print(f"  {service.__class__.__name__}: ", end="")
    print(f"implements send() and get_delivery_status()")

# Broadcast demonstrates all principles working together
print("\n--- Broadcasting (All Principles Together) ---")
manager.broadcast("System maintenance in 1 hour")

print("\n" + "=" * 60)
print("✓ All four OOP principles demonstrated successfully!")
print("=" * 60)

<a id='interview'></a>
## 6. Interview Questions & Answers

### Common OOP Interview Questions

### Q1: Explain the four pillars of OOP

**Answer:**
1. **Encapsulation**: Bundling data and methods, hiding implementation details
2. **Inheritance**: Creating new classes from existing ones, enabling code reuse
3. **Polymorphism**: Same interface, different implementations
4. **Abstraction**: Hiding complexity, showing only essential features

### Q2: What's the difference between `_var` and `__var` in Python?

**Answer:**
- `_var`: Protected (convention only, not enforced)
- `__var`: Private (name mangling to `_ClassName__var`)
- Neither truly prevents access, but `__var` makes it harder

### Q3: What is the diamond problem and how does Python solve it?

**Answer:**
Diamond problem occurs in multiple inheritance when a class inherits from two classes that share a common ancestor. Python solves this using C3 linearization (MRO - Method Resolution Order), ensuring:
- Each class appears only once in MRO
- Parent classes are called in a consistent order
- `super()` follows MRO, not just direct parent

### Q4: When to use composition vs inheritance?

**Answer:**
- **Inheritance**: "is-a" relationship (Dog is an Animal)
- **Composition**: "has-a" relationship (Car has an Engine)
- **Prefer composition** when:
  - No clear "is-a" relationship
  - Need flexibility to change behavior at runtime
  - Want to avoid tight coupling
- **Use inheritance** when:
  - Clear "is-a" relationship
  - Want to leverage polymorphism
  - Sharing common interface/behavior

### Q5: What is duck typing?

**Answer:**
"If it walks like a duck and quacks like a duck, it's a duck."
- Python focuses on object behavior, not type
- No need for explicit interfaces
- Objects compatible if they have required methods
- Example: Any object with `__iter__` and `__next__` is an iterator

### Q6: Explain `__str__` vs `__repr__`

**Answer:**
- `__str__`: User-friendly string for `str()` and `print()`
- `__repr__`: Developer-friendly, should be unambiguous, ideally `eval(repr(obj)) == obj`
- If only one is defined, use `__repr__`
- `__repr__` is fallback for `__str__`

### Q7: What are abstract base classes and when to use them?

**Answer:**
ABCs define interfaces that must be implemented by subclasses.
- Use when: Creating frameworks, defining protocols, ensuring implementation
- Cannot instantiate abstract classes
- Subclasses must implement all `@abstractmethod` methods
- Helps catch errors at instantiation time

### Q8: What is the Liskov Substitution Principle?

**Answer:**
Objects of a superclass should be replaceable with objects of a subclass without breaking the application.
- Subclass must honor parent's contract
- No strengthening preconditions
- No weakening postconditions
- Example: If `Bird.fly()` is expected, `Penguin` should not inherit `Bird` if it can't fly

## Practice Exercises

### Exercise 1: Encapsulation
Create a `BankAccount` class with:
- Private balance
- Methods: deposit, withdraw, get_balance
- Property for account_number (read-only)
- Transaction history

### Exercise 2: Inheritance
Create a vehicle hierarchy:
- Base class: `Vehicle` (brand, model, year)
- Subclasses: `Car`, `Motorcycle`, `Truck`
- Each with specific attributes and methods

### Exercise 3: Polymorphism
Create a `Shape` hierarchy with:
- Base class with area() and perimeter()
- Subclasses: Circle, Rectangle, Triangle
- Function that calculates total area of mixed shapes

### Exercise 4: Abstraction
Create an abstract `DataStorage` interface with:
- Methods: save(), load(), delete()
- Implementations: FileStorage, DatabaseStorage, CloudStorage

### Exercise 5: Integration
Build a simple e-commerce system using all principles:
- Abstract Product class
- Different product types (Physical, Digital)
- Shopping cart with encapsulated data
- Multiple payment methods (polymorphism)

## Summary

### The Four Pillars

| Principle | Purpose | Key Concepts |
|-----------|---------|-------------|
| **Encapsulation** | Data hiding, controlled access | `@property`, private attributes, getters/setters |
| **Inheritance** | Code reuse, hierarchies | `super()`, MRO, method overriding |
| **Polymorphism** | Flexible interfaces | Duck typing, operator overloading, magic methods |
| **Abstraction** | Hide complexity | ABC, `@abstractmethod`, interfaces |

### Python-Specific Features
- Duck typing over strict type checking
- Properties for encapsulation
- Multiple inheritance with MRO
- Magic methods for polymorphism
- ABC module for abstraction

### Best Practices
1. Use `@property` for Pythonic encapsulation
2. Favor composition over inheritance
3. Follow SOLID principles
4. Use duck typing appropriately
5. Keep hierarchies shallow
6. Document public APIs clearly

### Resources
- [Python Official Docs - Classes](https://docs.python.org/3/tutorial/classes.html)
- [Python Official Docs - ABC](https://docs.python.org/3/library/abc.html)
- [PEP 8 - Style Guide](https://pep8.org/)
- [Design Patterns in Python](https://refactoring.guru/design-patterns/python)
- [SOLID Principles](https://en.wikipedia.org/wiki/SOLID)